### 朴素贝叶斯分类器：
"朴素"：
- 各个特征之间的分布互相独立
- 各个特征具有同等的重要性

### 条件概率：
<div  align="center">    
<img src="images/%E6%9D%A1%E4%BB%B6%E6%A6%82%E7%8E%87%E5%85%AC%E5%BC%8F.png" width = "200" height = "200" alt="图片名称" align=center />
</div>

### 贝叶斯定理：
<div  align="center">    
<img src="images/%E8%B4%9D%E5%8F%B6%E6%96%AF%E5%AE%9A%E7%90%86.png" width = "230" height = "200" alt="图片名称" align=center />
</div>

### 贝叶斯分类器通常有两种实现方式
- 基于多项式模型：考虑特征在出现的频数
- 基于伯努利模型：不考虑特征出现的频数，假设特征等权重

In [69]:
import numpy as np

def loadDataSet():
    #训练数据集
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

In [70]:
original_X,original_y=loadDataSet()

In [71]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

In [72]:
### 生成基础词向量
myvoclist=createVocabList(original_X)
print myvoclist
print len(myvoclist)

['cute', 'love', 'help', 'garbage', 'quit', 'I', 'problems', 'is', 'park', 'stop', 'flea', 'dalmation', 'licks', 'food', 'not', 'him', 'buying', 'posting', 'has', 'worthless', 'ate', 'to', 'maybe', 'please', 'dog', 'how', 'stupid', 'so', 'take', 'mr', 'steak', 'my']
32


In [73]:
def setOfWords2Vec(vocabList, inputSet):
    #针对某条数据生成词向量
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print "the word: %s is not in my Vocabulary!" % word
    return returnVec

In [74]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords); p1Num = ones(numWords)      #change to ones() 
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    print p1Num,p1Denom
    print "-"*90
    print p0Num,p0Denom
    p1Vect = log(p1Num/p1Denom)          #change to log()
    p0Vect = log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [75]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult    
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)   # log(a.b)=log(a)+log(b)
    if p1 > p0:
        return 1
    else: 
        return 0
    

In [93]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    print array(trainMat)
    print "------"*20
    print array(listClasses)
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb)

In [94]:
testingNB()

[[0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0]
 [1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0]]
------------------------------------------------------------------------------------------------------------------------
[0 1 0 1 0 1]
[ 1.  1.  1.  2.  2.  1.  1.  1.  2.  2.  1.  1.  1.  2.  2.  2.  2.  2.
  1.  3.  1.  2.  2.  1.  3.  1.  4.  1.  2.  1.  1.  1.] 21.0
------------------------------------------------------------------------------------------
[ 2.  2.  2.  1.  1.  2.  2.  2.  1.  2.  2.  2.  2.  1.  1.  3.  1.  1.
  2.  1.  2.  2.  1.  2.  2.  2.  1.  2.  1.  2.  2.  4.] 26.0
['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1
